## Exploring fluxes

In [3]:
import cobra
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

In [2]:
model = read_sbml_model('model_atocopherol.xml')
model.objective = model.reactions.DM_avite1_c

To get a better understanding of how to improve the production pathway, the associated reactions and fluxes are explored. The metabolite 3-(4-Hydroxyphenyl)pyruvate is the starting point for the heterologous pathway that was added to the model.

In [85]:
model.metabolites.get_by_id('34hpp_c')

Metabolite identifier,34hpp_c
Name,3-(4-Hydroxyphenyl)pyruvate
Memory address,0x07ff88828e4f0
Formula,C9H7O4
Compartment,c
In 3 reaction(s),"TYRTA, 34HPPOR, PPND"


3-(4-Hydroxyphenyl)pyruvate is part of three reactions. 34HPPOR is the first reaction of the heterologous pathway that consumes 3-(4-Hydroxyphenyl)pyruvate. Whereas TYRTA and PPND are reactions where 3-(4-Hydroxyphenyl)pyruvate is produced.

In [71]:
model.reactions.PPND

Reaction identifier,PPND
Name,Prephenate dehydrogenase
Memory address,0x07ff889913df0
Stoichiometry,nad_c + pphn_c --> 34hpp_c + co2_c + nadh_c Nicotinamide adenine dinucleotide + Prephenate --> 3-(4-Hydroxyphenyl)pyruvate + CO2 CO2 + Nicotinamide adenine dinucleotide - reduced
GPR,b2600
Lower bound,0.0
Upper bound,1000.0


In [86]:
model.reactions.TYRTA

Reaction identifier,TYRTA
Name,Tyrosine transaminase
Memory address,0x07ff88a2b7880
Stoichiometry,akg_c + tyr__L_c <=> 34hpp_c + glu__L_c 2-Oxoglutarate + L-Tyrosine <=> 3-(4-Hydroxyphenyl)pyruvate + L-Glutamate
GPR,b4054 or b0928 or b3770
Lower bound,-1000.0
Upper bound,1000.0


While exploring the fluxes, the production of biomass is set to half of the optimal production rate as a compromise between cell growth and production of alpha-tocopherol.

In [80]:
with model:
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.44
    solution = model.optimize()
print("EX_tyr__L_e", solution.fluxes.EX_tyr__L_e)
print("PPND", solution.fluxes.PPND)
print("TYRTA",solution.fluxes.TYRTA)
print("HGPHT",solution.fluxes.HGPHT)
print("TOCOPHOM1",solution.fluxes.TOCOPHOM1)
print("DM_avite1_c",solution.fluxes.DM_avite1_c)

EX_tyr__L_e 0.0
PPND 0.6741627607421248
TYRTA -0.06077236
HGPHT 0.6133904007421248
TOCOPHOM1 0.6133904007421248
DM_avite1_c 0.6133904007421248


In [58]:
with model:
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.44
    model.reactions.EX_tyr__L_e.lower_bound = -10
    solution = model.optimize()
print("EX_tyr__L_e", solution.fluxes.EX_tyr__L_e)
print("PPND", solution.fluxes.PPND)
print("TYRTA",solution.fluxes.TYRTA)
print("TOCOPHOM1",solution.fluxes.TOCOPHOM1)
print("DM_avite1_c",solution.fluxes.DM_avite1_c)

EX_tyr__L_e -0.8597443803366394
PPND 0.0
TYRTA 0.7989720203366394
TOCOPHOM1 0.7989720203366394
DM_avite1_c 0.7989720203366394


It is shown here that in the absence of exogenous tyrosine, 3-(4-Hydroxyphenyl)pyruvate is produced in reaction PPND while reaction TYRTA has a negative flux meaning that a part of the 3-(4-Hydroxyphenyl)pyruvate pool is consumed to produce tyrosine. When tyrosine is supplied in the medium, the reaction PPND does not carry any flux anymore. In this case, 3-(4-Hydroxyphenyl)pyruvate is produced from tyrosine in the reaction TYRTA. For this reason, TYRTA could be a potential target for overexpresssion. To test this, a flux variability analysis is performed which shows the minimal and maximal flux that a reaction can carry.

In [13]:
from cobra.flux_analysis import flux_variability_analysis

with model:
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.44
    model.reactions.EX_tyr__L_e.lower_bound = -10
    reactions_OE = [model.reactions.EX_tyr__L_e, model.reactions.TYRTA, model.reactions.PPND, model.reactions.DM_avite1_c]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
print(fva)

              minimum   maximum
EX_tyr__L_e -1.331703 -0.488684
TYRTA        0.427912  0.798973
PPND         0.000000  0.291163
DM_avite1_c  0.719075  0.798973


In an attempt to enforce a higher flux in the reaction TYRTA, the lower bound was raised to 0.8. However, this modification proofed to be infeasible. 

In [88]:
with model:
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.44
    model.reactions.EX_tyr__L_e.lower_bound = -10
    model.reactions.TYRTA.lower_bound = 0.8
    reactions_OE = [model.reactions.EX_tyr__L_e, model.reactions.TYRTA, model.reactions.PPND, model.reactions.DM_avite1_c]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
print(fva)

Infeasible: There is no optimal solution for the chosen objective! (infeasible).

In [89]:
with model:
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.44
    model.reactions.EX_tyr__L_e.lower_bound = -10
    model.reactions.TYRTA.lower_bound = 0.7
    reactions_OE = [model.reactions.EX_tyr__L_e, model.reactions.TYRTA, model.reactions.PPND, model.reactions.DM_avite1_c]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
print(fva)

              minimum   maximum
EX_tyr__L_e -1.331703 -0.760772
TYRTA        0.700000  0.798973
PPND         0.000000  0.077662
DM_avite1_c  0.719075  0.798973


Changing the lower bound of TYRTA to 0.7 to enforce a higher flux did not change the maximal flux of alpha-tocopherol production. 

### Model summary

The summary() function is used to get an overview of the secretion reactions. Pathways that lead to the secretion of a metabolite that is in competition with the production of the desired product are potential targets for a knockout or a downregulation. To the secretion of the wildtype *E. coli* model is compared with the modiefied model containing the pathway of alpha-tocopherol production and the addition of tyrosine to the medium.

#### Summary of wildtype model

In [5]:
model_1 = read_sbml_model('iML1515.xml')

In [6]:
model_1.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.004565,0,0.00%
cl_e,EX_cl_e,0.004565,0,0.00%
cobalt2_e,EX_cobalt2_e,2.192E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006218,0,0.00%
fe2_e,EX_fe2_e,0.01409,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
k_e,EX_k_e,0.1712,0,0.00%
mg2_e,EX_mg2_e,0.007608,0,0.00%
mn2_e,EX_mn2_e,0.000606,0,0.00%
mobd_e,EX_mobd_e,6.139E-06,0,0.00%


In [7]:
model_1.metabolites.meoh_e

Metabolite identifier,meoh_e
Name,Methanol
Memory address,0x07fa606710dc0
Formula,CH4O1
Compartment,e
In 2 reaction(s),"EX_meoh_e, MEOHtex"


#### Summary of modified model

In [28]:
model_2 = read_sbml_model('model_atocopherol_thr.xml')

In [29]:
model_2.objective = model_2.reactions.DM_avite1_c

In [30]:
with model_2:
    model_2.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.44
    summary = model_2.summary()
summary

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.00229,0,0.00%
cl_e,EX_cl_e,0.00229,0,0.00%
cobalt2_e,EX_cobalt2_e,1.1E-05,0,0.00%
cu2_e,EX_cu2_e,0.000312,0,0.00%
fe2_e,EX_fe2_e,0.007067,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
k_e,EX_k_e,0.08588,0,0.00%
mg2_e,EX_mg2_e,0.003817,0,0.00%
mn2_e,EX_mn2_e,0.000304,0,0.00%
mobd_e,EX_mobd_e,3.08E-06,0,0.00%
